# Tests for the Dilithium firmware

In [1]:
import logging
logging.basicConfig(level=logging.NOTSET)
logging.getLogger('io.github.alex1s.python-dilithium').setLevel(logging.WARNING)

we append the path so that we can use the chipwhisperer library

In [2]:
import sys
if '../../../software' not in sys.path:
    sys.path.append('../../../software')
sys.path

['/home/alexis/chipwhisperer.dilithium/jupyter/courses/fault101',
 '/usr/lib/python38.zip',
 '/usr/lib/python3.8',
 '/usr/lib/python3.8/lib-dynload',
 '',
 '/home/alexis/.venvs/chipwhisperer.dilithium/lib/python3.8/site-packages',
 '../../../software']

In [3]:
import logging
import chipwhisperer as cw
import importlib
dilithium = importlib.import_module("python-dilithium.dilithium")
dilithium.generic = importlib.import_module("python-dilithium.dilithium.generic")
logging.basicConfig(level=logging.NOTSET)
cw

<module 'chipwhisperer' from '/home/alexis/chipwhisperer.dilithium/jupyter/courses/fault101/../../../software/chipwhisperer/__init__.py'>

In [4]:
SCOPETYPE = 'OPENADC'
PLATFORM = 'CW308_STM32F4'
SS_VER = 'SS_VER_2_1'

NIST_SECURITY_LEVEL = 2
HIGH_SPEED = False

In [5]:
%run "../../Setup_Scripts/Setup_Generic_Dilithium.ipynb"

(ChipWhisperer NAEUSB WARNING|File naeusb.py:713) Your firmware (0.30) is outdated - latest is 0.64 See https://chipwhisperer.readthedocs.io/en/latest/firmware.html for more information
DEBUG:SimpleSerial2Dilithium:SimpleSerial2Dilithium logger says hello!


INFO: Found ChipWhisperer😍


In [6]:
%%bash -s "$PLATFORM" "$SS_VER"
cd ../../../hardware/victims/firmware/simpleserial-dilithium-ref
make PLATFORM=$1 CRYPTO_TARGET=NONE CRYPTO_OPTIONS=NONE SS_VER=$2

DEBUG:asyncio:Using selector: EpollSelector


SS_VER set to SS_VER_2_1
SS_VER set to SS_VER_2_1
make[1]: '.dep' is up to date.
SS_VER set to SS_VER_2_1
.
Welcome to another exciting ChipWhisperer target build!!
arm-none-eabi-gcc (15:9-2019-q4-0ubuntu1) 9.2.1 20191025 (release) [ARM/arm-9-branch revision 277599]
Copyright (C) 2019 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.

.
Compiling:
-en     simpleserial-dilithium-ref.c ...
mkdir -p "objdir-CW308_STM32F4"
-e Done!
.
Compiling:
-en     dilithium/ref/sign.c ...
mkdir -p "objdir-CW308_STM32F4/dilithium/ref"
-e Done!
.
Compiling:
-en     .././simpleserial/simpleserial.c ...
mkdir -p "objdir-CW308_STM32F4"
-e Done!
.
Compiling:
-en     .././hal/stm32f4/stm32f4_hal.c ...
mkdir -p "objdir-CW308_STM32F4"


In file included from .././hal/stm32f4/stm32f4_hal.c:3:
.././hal/stm32f4/stm32f4_hal_lowlevel.h:108: warning: "STM32F415xx" redefined
  108 | #define STM32F415xx
      | 
<command-line>: note: this is the location of the previous definition


-e Done!
.
Compiling:
-en     .././hal/stm32f4/stm32f4_hal_lowlevel.c ...
mkdir -p "objdir-CW308_STM32F4"


In file included from .././hal/stm32f4/stm32f4_hal_lowlevel.c:39:
.././hal/stm32f4/stm32f4_hal_lowlevel.h:108: warning: "STM32F415xx" redefined
  108 | #define STM32F415xx
      | 
<command-line>: note: this is the location of the previous definition


-e Done!
.
Compiling:
-en     .././hal/stm32f4/stm32f4_sysmem.c ...
mkdir -p "objdir-CW308_STM32F4"
-e Done!
.
Compiling:
-en     .././hal/stm32f4/stm32f4xx_hal_rng.c ...
mkdir -p "objdir-CW308_STM32F4"
-e Done!
.
Assembling: .././hal/stm32f4/stm32f4_startup.S
arm-none-eabi-gcc -c -mcpu=cortex-m4 -I. -x assembler-with-cpp -mthumb -mfloat-abi=soft -fmessage-length=0 -ffunction-sections -DF_CPU=7372800 -Wa,-gstabs,-adhlns=objdir-CW308_STM32F4/stm32f4_startup.lst -I.././simpleserial/ -I.././hal -I.././hal/stm32f4 -I.././hal/stm32f4/CMSIS -I.././hal/stm32f4/CMSIS/core -I.././hal/stm32f4/CMSIS/device -I.././hal/stm32f4/Legacy -I.././crypto/ .././hal/stm32f4/stm32f4_startup.S -o objdir-CW308_STM32F4/stm32f4_startup.o
.
LINKING:
-en     simpleserial-dilithium-ref-CW308_STM32F4.elf ...
-e Done!
.
Creating load file for Flash: simpleserial-dilithium-ref-CW308_STM32F4.hex
arm-none-eabi-objcopy -O ihex -R .eeprom -R .fuse -R .lock -R .signature simpleserial-dilithium-ref-CW308_STM32F4.elf simples

In [9]:
fw_path = "../../../hardware/victims/firmware/simpleserial-dilithium-ref/simpleserial-dilithium-ref-{}.hex".format(PLATFORM)
cw.program_target(scope, prog, fw_path)
target.reset_comms()

Detected known STMF32: STM32F40xxx/41xxx
Extended erase (0x44), this can take ten seconds or more
Attempting to program 21947 bytes at 0x8000000
STM32F Programming flash...
STM32F Reading flash...
Verified flash OK, 21947 bytes


In [10]:
if HIGH_SPEED:
    scope.clock.clkgen_freq = 24E6
    target.baud = 230400*24/7.37

In [11]:
dilithium.generic.pseudorandombytes_seed(b'attack-shuffling-countermeasure-keypair')
pk, sk = dilithium.generic.keypair(nist_security_level=NIST_SECURITY_LEVEL)
sk.hex()

'1f40eda0c21251c5589d865df25fe4178e9e4994ed337021b56494b0323d84e3b9a21afe7320fe9c3a7f2252fad47e87c43cc06d2d004783e20433fe82e5bc78103fb0eda5e28396ef7621042eef0e2d48a1b132a0a7d256ae97ee6001501b550ab81104860844260a19a7308222910c256a1c1044a11886c4c42501a309118221d0862dc3322d128544d8106811c13192946c983002981828890691593041dc004c54b680642249998271421892da42054cc250143486a49021e0c82d1b2564d19209814480db2626504208db828d6488411ba828d0b820c1c40021c92160420a1a41091b08210a40109c488588020c410621022762d014282323324040040082011cc3506004810b814cccc48d8bc281e23022d0402d80a085ca4891e2a44054328a9a1232582608d8146e1b170048120d2317268c34850bb84410472a12842c49042449280858266824934d1bc0690b24501a1126a118301a090d512211c34251002430c4064a89c80851328e029868904689134366531081003826583652daa4011a140d038910c9368262064204a350e02025d4c270c1c07193862810278e00048ad8262c1b316109906153022c01404601920199a04c02234902a348e3924883486e4300469ab28503b1800b4569e1089052b269c104440b46520c840d03302e4304501c9660420492d43270211804a33

In [12]:
msg = b'hi'
msg

b'hi'

In [13]:
signature = dilithium.generic.signature(msg, sk, nist_security_level=NIST_SECURITY_LEVEL)
signature

b'/\xfb\x9f\x9d5\xc0b_\xb3y\xbby\xc8\xfbi\x7f\xf2\xa1\xab,\xf8\x1c{\x10\xbf\x87tC\xd4\xf7\xd0M\xc5"\x0f\xae\xfey\x82\xfeZ\xcf\x1c\xbe\x0c\x1d\x12j\x87\xe2\xb1\x1d\xf5\xafbo\xd2`l\x11\xcf\x04\xc5lTy\xccY\n\x13\xed"#\xd6y\xb7\xcf|\x1c\xa4\xef\x80\xf8\x18%-\xec\xfb\xbf?\x13,\x84\x14+ !\xd3\x13\x9b\x8e\xf3\xf2\xe6l\xf7\xe1\x92%\x90\x06.\xc3\xce=\x84#@\xab\x06\xdf\x91\t\xbe\xd9\xe8;L_-\x91]\x81\xa3\xa7\xb4\xcd\xce\xb7\x11\x8d\xb5\xa4\x99#\xd2\xb5\xcd\x00F\xddY-\x9b\x88\xf0F\xdf\xeeg\xb9\x07\x1eJ\x1b\x0b\xd3\x88\x01^\xaa\x9d\xf1>\x198\xf7\x12\xc9\x0e\'`\xde\xfd[\xd5\xe9\xf6"\xe60\xb9\n\x16\x98\x01\xce\x08H\xed\x91\xf2\x8a\xf8%\x9c\xb5+"\x7f;\xe7\x91s\xf1\x1a\xc7Ak4\xd5qX1\x9aB\x98\x19\xcf\xd5h\xa5D=\x8aK9QK\x94\x8b\xe5Wb_/\xc4\xaa\x01"\xad\xdf7\x0fvY\x1d\x01\xd9.\xe0:\xb7\xc6\xb2\xbd8\tg\x8a\x1b1\xf9\xe1\xb4\x80Y\xecw\x10GEG\t\xa5\xd2\xbe\x06\xdf\x88\x0e\\\xb2\xb5\n\xc2\x95\xd4\xdc1Z!\xb8\x8f\xe3\xc2\xc4U\xf9\xb4{f\xbc\x1e\xf3\xee\x19+n\xed\xbe\x10/\x91>\xe3\xfb\xaa\x0f\xef\xe3\xa6,\x95\xe7\

In [14]:
target.reboot_flush()
try:
    target.sign(msg)
except cw.targets.TargetIOError as e:
    print(e.data)
    raise e

In [15]:
target_sig = target.get_sig()
target_sig

b'/\xfb\x9f\x9d5\xc0b_\xb3y\xbby\xc8\xfbi\x7f\xf2\xa1\xab,\xf8\x1c{\x10\xbf\x87tC\xd4\xf7\xd0M\xc5"\x0f\xae\xfey\x82\xfeZ\xcf\x1c\xbe\x0c\x1d\x12j\x87\xe2\xb1\x1d\xf5\xafbo\xd2`l\x11\xcf\x04\xc5lTy\xccY\n\x13\xed"#\xd6y\xb7\xcf|\x1c\xa4\xef\x80\xf8\x18%-\xec\xfb\xbf?\x13,\x84\x14+ !\xd3\x13\x9b\x8e\xf3\xf2\xe6l\xf7\xe1\x92%\x90\x06.\xc3\xce=\x84#@\xab\x06\xdf\x91\t\xbe\xd9\xe8;L_-\x91]\x81\xa3\xa7\xb4\xcd\xce\xb7\x11\x8d\xb5\xa4\x99#\xd2\xb5\xcd\x00F\xddY-\x9b\x88\xf0F\xdf\xeeg\xb9\x07\x1eJ\x1b\x0b\xd3\x88\x01^\xaa\x9d\xf1>\x198\xf7\x12\xc9\x0e\'`\xde\xfd[\xd5\xe9\xf6"\xe60\xb9\n\x16\x98\x01\xce\x08H\xed\x91\xf2\x8a\xf8%\x9c\xb5+"\x7f;\xe7\x91s\xf1\x1a\xc7Ak4\xd5qX1\x9aB\x98\x19\xcf\xd5h\xa5D=\x8aK9QK\x94\x8b\xe5Wb_/\xc4\xaa\x01"\xad\xdf7\x0fvY\x1d\x01\xd9.\xe0:\xb7\xc6\xb2\xbd8\tg\x8a\x1b1\xf9\xe1\xb4\x80Y\xecw\x10GEG\t\xa5\xd2\xbe\x06\xdf\x88\x0e\\\xb2\xb5\n\xc2\x95\xd4\xdc1Z!\xb8\x8f\xe3\xc2\xc4U\xf9\xb4{f\xbc\x1e\xf3\xee\x19+n\xed\xbe\x10/\x91>\xe3\xfb\xaa\x0f\xef\xe3\xa6,\x95\xe7\

In [16]:
assert len(target_sig) == len(signature)
if target_sig == signature:
    print('YES; IT WORKS')
else:
    print('len ok but ah shit, check again')

YES; IT WORKS


In [19]:
# found using
# target.filter_msgs_one_iter([struct.pack('!H', i) for i in range(10)])
msgs_no_retry = [b'\x00\x01', b'\x00\x02', b'\x00\x07', b'\x00\t']
the_msg = msgs_no_retry[0]

In [20]:
def dilithium2_polyz_unpack_settings():
    scope.default_setup()
    scope.cglitch_setup() # default_setup for clock glitching
    scope.adc.samples = 24400 # the maximum for CWLite

def get_offset_first_iter():
    dilithium2_polyz_unpack_settings()
    target.reboot_flush()
    scope.arm()
    target.sign(msgs_no_retry[0])
    timed_out = scope.capture()
    assert not timed_out
    trig_count = scope.adc.trig_count
    # we have two coeffs (lvl2) per iteration and 256 coeffs in total, resulting in 128 iterations
    print(f'time per iteration: {trig_count / 64}')
    return trig_count / 64


def get_correct_sig() -> bytes:
    """returns something around 500"""
    target.sign(msgs_no_retry[0])
    sig = target.get_sig()
    return sig

print(get_offset_first_iter())
correct_sig = get_correct_sig()

DEBUG:root:Trigger found at 98


time per iteration: 998.4375
998.4375


In [ ]:
failhere

In [ ]:
reboot_flush()
scope.arm()
target.simpleserial_write("g", bytearray([]))
scope.capture()
val = target.simpleserial_read_witherrors('r', 4, glitch_timeout=10)#For loop check
valid = val['valid']
if valid:
    response = val['payload']
    raw_serial = val['full_response']
    error_code = val['rv']

print(val)

In [ ]:
import chipwhisperer.common.results.glitch as glitch
gc = glitch.GlitchController(groups=["success", "reset", "normal"], parameters=["width", "offset", "ext_offset"])
gc.display_stats()

In [ ]:
if scope._is_husky:
    scope.glitch.enabled = True

scope.glitch.clk_src = "clkgen" # set glitch input clock
scope.glitch.output = "glitch_only" # glitch_out = clk ^ glitch
scope.glitch.trigger_src = "ext_single" # glitch only after scope.arm() called
if PLATFORM == "CWLITEXMEGA":
    scope.io.glitch_lp = True
    scope.io.glitch_hp = True
elif PLATFORM == "CWLITEARM":
    scope.io.glitch_lp = True
    scope.io.glitch_hp = True
elif PLATFORM == "CW308_STM32F3":
    scope.io.glitch_hp = True
    scope.io.glitch_lp = True

Some tips for finding good glitches:

1. There's a lot of stuff fighting our glitch this time - unlike the clock line, the Vcc rail isn't supposed to oscillate! As such shorter glitches will have no effect. A good strategy can often to be to increase the minimum glitch width until you start seeing consistant crashes, then backing off on the width.
1. The repeat parameter behaves very differently than with voltage glitching - at the boosted clock rate, the Vcc often won't recover before the next glitch. Try different repeat values as well.
1. We've built in a success/reset measurement into the glitch loop. Once you've found some glitch spots, this will help you evaluate which ones are best for your target.

It can take a very long time to do go through the full search space, so you may want to stop after you get a certain number of succeses. By default here, it will be 1, but you may want to change it to 10, 20, or go even higher.

In [ ]:
TOTAL_SUCCESSES = 1

In [ ]:
from importlib import reload
import chipwhisperer.common.results.glitch as glitch
from tqdm.notebook import trange
import struct

#disable logging
cw.set_all_log_levels(cw.logging.CRITICAL)

# These width/offset settings are for CW-Lite/Pro; width/offset are expressed differently for Husky (see Fault 1_1)
g_step = 0.4
if PLATFORM=="CWLITEXMEGA":
    gc.set_range("width", 43.5, 47.8)
    gc.set_range("offset", -48, -10)
    gc.set_range("ext_offset", 7, 10)
    scope.glitch.repeat = 11
elif PLATFORM == "CWLITEARM":
    #should also work for the bootloader memory dump
    gc.set_range("width", 34 - 10, 36 + 10)
    gc.set_range("offset", -40, 10 + 30)
    gc.set_range("ext_offset", 6 - 1, 7 + 1)
    scope.glitch.repeat = 7
elif PLATFORM == "CW308_STM32F3":
    #these specific settings seem to work well for some reason
    #also works for the bootloader memory dump
    gc.set_range("ext_offset", 9, 12)
    gc.set_range("width", 47.6, 49.6)
    gc.set_range("offset", -19, -21.5)
    scope.glitch.repeat = 5

gc.set_global_step(g_step)

scope.adc.timeout = 0.5

reboot_flush()
sample_size = 10
loff = scope.glitch.offset
lwid = scope.glitch.width
total_successes = 0
for glitch_setting in gc.glitch_values():
    scope.glitch.offset = glitch_setting[1]
    scope.glitch.width = glitch_setting[0]
    scope.glitch.ext_offset = glitch_setting[2]
    #print(scope.glitch.ext_offset)
    successes = 0
    resets = 0
    for i in range(10):
        target.flush()
        if scope.adc.state:
            # can detect crash here (fast) before timing out (slow)
            #print("Trigger still high!")
            gc.add("reset", (scope.glitch.width, scope.glitch.offset, scope.glitch.ext_offset))

            #Device is slow to boot?
            reboot_flush()
            resets += 1
            
        scope.arm()
        
        #Do glitch loop
        target.simpleserial_write("g", bytearray([]))
        
        ret = scope.capture()
        
        
        
        
        scope.io.glitch_hp = False
        scope.io.glitch_hp = True
        scope.io.glitch_lp = False
        scope.io.glitch_lp = True
        if ret:
            #print('Timeout - no trigger')
            gc.add("reset", (scope.glitch.width, scope.glitch.offset, scope.glitch.ext_offset))
            resets += 1

            #Device is slow to boot?
            reboot_flush()

        else:
            val = target.simpleserial_read_witherrors('r', 4, glitch_timeout=10, timeout=50)#For loop check
            if val['valid'] is False:
                gc.add("reset", (scope.glitch.width, scope.glitch.offset, scope.glitch.ext_offset))
                reboot_flush()
                resets += 1
                #print(val)
            else:
                gcnt = struct.unpack("<I", val['payload'])[0]
                
                if gcnt != 2500: #for loop check
                    gc.add("success", (scope.glitch.width, scope.glitch.offset, scope.glitch.ext_offset))
                    print((scope.glitch.width, scope.glitch.offset, scope.glitch.ext_offset))
                    successes += 1
                else:
                    gc.add("normal", (scope.glitch.width, scope.glitch.offset, scope.glitch.ext_offset))
    if successes > 0:                
        print("successes = {}, resets = {}, offset = {}, width = {}, ext_offset = {}".format(successes, resets, scope.glitch.offset, scope.glitch.width, scope.glitch.ext_offset))
        total_successes += successes
    if total_successes > TOTAL_SUCCESSES: #increase if you want to check more locations
        break
print("Done glitching")
#enable logging
cw.set_all_log_levels(cw.logging.WARNING)

In [ ]:
%matplotlib notebook
gc.results.plot_2d(plotdots={"success":"+g", "reset":"xr", "normal":None})

In [21]:
scope.dis()
target.dis()

In [ ]:
assert total_successes >= 1